In [68]:
#Importowanie bibliotek

#komendy do importu (windows):
#pip install pycryptodomex --no-binary :all:
#pip install pyspectator
#pip install wmi

#komendy do importu (linux):
#pip install pycryptodomex
#pip install pyspectator
#pip install wmi (niepotrzebne)

from pyspectator.computer import Cpu
import wmi
from Crypto.Hash import SHA1
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15

In [69]:
#Kod generujący liczbę losową
#UWAGA: Na systemach windows należy mieć zainstalowany i uruchomiony Open Hardware Monitor - https://openhardwaremonitor.org/downloads/
#W celu pobrania wartości temperatury. Na systemach linux można zakomentować linijkę temp = getWindowsTemp i odkomentować temp = int(CPU.temperature)

_w_ = wmi.WMI(namespace="root/OpenHardwareMonitor")

def getWindowsTemp():
    sensors = _w_.Sensor()
    for sensor in sensors:
        if sensor.SensorType==u'Temperature' and not 'GPU' in sensor.Name:
            return int(sensor.Value)
    return 0

def myTRNG(SIZE = 1024):
    SIZE = SIZE * 8
    enable = True
    actualBitStream = ''
    while(enable):
        CPU = Cpu(monitoring_latency=1)  
        seed = int(CPU.load)
        temp = getWindowsTemp()
        actualSeed = ''
        #temp = int(CPU.temperature)
        for i in range(7):
            seed = (13*int(seed)+3*temp)%256    
            actualSeed += str(bin(seed))[2:]
        if(len(actualBitStream) >= SIZE):
            actualBitStream = actualBitStream[0:SIZE]
            enable = False
        else :
            actualBitStream += actualSeed
    return int(actualBitStream,2).to_bytes(((len(actualBitStream) + 7)) //8,'big')

In [70]:
#Funkcja zwracająca jednocześnie publiczny i prywatny klucz za pomocą biblioteki
def getPrivatePublicKey():
    key = RSA.generate(1024, myTRNG) #przekazujemy do funkcji długość ciągu bajtowego oraz funkcję geneurjącą
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

In [71]:
#RSA
#Generowanie prywatnego i publicznego klucza
priv_key, pub_key = getPrivatePublicKey()

In [72]:
#zapisanie kluczy do plików
priv_key_file = open("priv_key.pem", "wb")
priv_key_file.write(priv_key)
priv_key_file.close()

pub_key_file = open("pub_key.pem", "wb")
pub_key_file.write(pub_key)
pub_key_file.close()

In [73]:
#Wczytuje wartości publicznego i prywatnego klucza z pliku
# Load the private key from the file
with open('priv_key.pem', 'rb') as f:
    to_use_priv_key = RSA.import_key(f.read())

# Load the public key from the file
with open('pub_key.pem', 'rb') as f:
    to_use_pub_key = RSA.import_key(f.read())

Private RSA key at 0x289B18B7160
Public RSA key at 0x289B18B7F70


In [76]:
#wczytaj plik, utwórz jego hash i utwórz podpis. Zapisz podpis do pliku
with open('toSign.txt', 'rb') as f:
    to_sign = f.read()
    
hash_obj = SHA1.new(to_sign)
# tworzy podpis za pomocą prywatnego klucza
signature = pkcs1_15.new(to_use_priv_key).sign(hash_obj)

signature_file = open("signature.sig", "wb")
signature_file.write(signature)
signature_file.close()

In [77]:
#Sprawdź poprawność kluczy
with open('toSign.txt', 'rb') as f:
    to_sign = f.read()
    
hash_obj = SHA1.new(to_sign)
with open('signature.sig', 'rb') as f:
    signature = f.read()

try:
    pkcs1_15.new(to_use_pub_key).verify(hash_obj, signature)
    print("Signature is valid.")
except (ValueError, TypeError):
    print("Signature is not valid.")

Signature is valid.
